In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import glob
import PIL
from pathlib import Path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
import plotly.express as px

In [8]:
image= Path('../input/marvel-heroes/marvel')

In [9]:
file = list(image.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file))

file = pd.Series(file, name='File').astype(str)
labels = pd.Series(labels, name='Label')

df = pd.concat([file, labels], axis=1)

samples = []
for category in df['Label'].unique():
    category_slice = df.query("Label == @category")
    samples.append(category_slice.sample(700, random_state=1))
df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

train, test = train_test_split(df, test_size=0.1, shuffle=True, random_state=1)

In [10]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [11]:
train_images = train_gen.flow_from_dataframe(
    dataframe=train,
    x_col='File',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_gen.flow_from_dataframe(
    dataframe=train,
    x_col='File',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_gen.flow_from_dataframe(
    dataframe=test,
    x_col='File',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 4032 validated image filenames belonging to 8 classes.
Found 1008 validated image filenames belonging to 8 classes.
Found 560 validated image filenames belonging to 8 classes.


In [12]:
pretrained_model = VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)

pretrained_model.trainable = False

inputs = pretrained_model.input
x = layers.Dense(128, activation='relu')(pretrained_model.output)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(8, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

80142336/80134624 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [13]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
126/126 [==============================] - 116s 846ms/step - loss: 3.8349 - accuracy: 0.1809 - val_loss: 1.7440 - val_accuracy: 0.3690
Epoch 2/100
126/126 [==============================] - 78s 617ms/step - loss: 1.7558 - accuracy: 0.3489 - val_loss: 1.4620 - val_accuracy: 0.5198
Epoch 3/100
126/126 [==============================] - 77s 613ms/step - loss: 1.5144 - accuracy: 0.4548 - val_loss: 1.3323 - val_accuracy: 0.5556
Epoch 4/100
126/126 [==============================] - 77s 611ms/step - loss: 1.3907 - accuracy: 0.5089 - val_loss: 1.2661 - val_accuracy: 0.6091
Epoch 5/100
126/126 [==============================] - 76s 605ms/step - loss: 1.2508 - accuracy: 0.5582 - val_loss: 1.1544 - val_accuracy: 0.6190
Epoch 6/100
126/126 [==============================] - 74s 587ms/step - loss: 1.1876 - accuracy: 0.5847 - val_loss: 1.1161 - val_accuracy: 0.6667
Epoch 7/100
126/126 [==============================] - 70s 553ms/step - loss: 1.1033 - accuracy: 0.6092 - val_loss: 1.0707 

In [14]:
results = model.evaluate(test_images)
print(results)
print("    Loss: {:.5f}".format(results[0]))
print("Accuracy: {:.3f}%".format(results[1] * 100))


18/18 [==============================] - 11s 648ms/step - loss: 0.6105 - accuracy: 0.8089
[0.6104844212532043, 0.8089285492897034]
    Loss: 0.61048
Accuracy: 80.893%


In [ ]:
model.save("")